# Keras MLP 

For this notebook the Reuters newswire dataset will be used. This dataset consists of 11,228 newswires from the Reuters news agency. Each wire is encoded as a sequence of word indexes, just as in the IMDB data we encountered in lecture 5 of this series. Moreover, each wire is categorised into one of 46 topics, which will serve as our label. This dataset is available through the Keras API.

We want to create a Multi-layer perceptron (MLP) using Keras which we can train to classify news items into the specified 46 topics.

We start by installing and importing everything we need.

In [1]:
!pip install tensorflow==2.2.0rc0

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 515.9 MB 26 kB/s s eta 0:00:01MB 18.8 MB/s eta 0:00:27     |████████▍                       | 135.5 MB 61.2 MB/s eta 0:00:07MB/s eta 0:00:07        | 217.7 MB 60.2 MB/s eta 0:00:05��▌               | 265.6 MB 59.3 MB/s eta 0:00:05     |██████████████████▉             | 303.6 MB 71.8 MB/s eta 0:00:03:00:03     |███████████████████████▋        | 381.2 MB 15.6 MB/s eta 0:00:09��█████████████▊    | 446.8 MB 70.5 MB/s eta 0:00:01
     |████████████████████████████████| 448 kB 48.8 MB/s eta 0:00:01
     |██

In [2]:
import tensorflow as tf
if not tf.__version__ == '2.2.0-rc0':
    print(tf.__version__)
    raise ValueError('please upgrade to TensorFlow 2.2.0-rc0, or restart your Kernel (Kernel->Restart & Clear Output)')

Keras' Sequential model has only two types of layers: Dense and Dropout. We also specify a random seed to make our results reproducible. Next, we load the Reuters data set.

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
seed = 1337
np.random.seed(seed)
from tensorflow.keras.datasets import reuters

max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2,
                                                         seed=seed)
num_classes = np.max(y_train) + 1  # 46 topics

2113536/2110848 [==============================] - 0s 0us/step


/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


Note that we cap the maximum number of words in a news item to 1000 by specifying the *num_words* key word. Also, 20% of the data will be test data and we ensure reproducibility by setting our random seed.

Our training features are still simply sequences of indexes and we need to further preprocess them, so that we can plug them into a *Dense* layer. For this we use a *Tokenizer* from Keras' text preprocessing module. This tokenizer will take an index sequence and map it to a vector of length *max_words=1000*. Each of the 1000 vector positions corresponds to one of the words in our newswire corpus. The output of the tokenizer has a 1 at the i-th position of the vector, if the word corresponding to i is in the description of the newswire, and 0 otherwise. Even if this word appears multiple times, we still just put a 1 into our vector, i.e. our tokenizer is binary. We use this tokenizer to transform both train and test features:

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

## Part 1 - Label encoding

Use to_categorical, as we did in the lectures, to transform both *y_train* and *y_test* into one-hot encoded vectors of length *num_classes*:

In [5]:
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

## Part 2 - Model definition

Next, initialise a Keras *Sequential* model and add three layers to it:

    Layer: Add a *Dense* layer with in input_shape=(max_words,), 512 output units and "relu" activation.
    Layer: Add a *Dropout* layer with dropout rate of 50%.
    Layer: Add a *Dense* layer with num_classes output units and "softmax" activation.

In [7]:
model = Sequential()
model.add(Dense(512, activation = 'relu', input_shape=(max_words,)))
model.add(Dropout(0.5))
model.add((Dense(num_classes, activation = 'softmax')))

## Part 3 - Model compilation

As the next step, we need to compile our Keras model with a training configuration. Compile your model with "categorical_crossentropy" as loss function, "adam" as optimizer and specify "accuracy" as evaluation metric. NOTE: In case you get an error regarding h5py, just restart the kernel and start from scratch

In [8]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])

## Part 4 - Model training and evaluation

Next, define the batch_size for training as 32 and train the model for 5 epochs on *x_train* and *y_train* by using the *fit* method of your model. Then calculate the score for your trained model by running *evaluate* on *x_test* and *y_test* with the same batch size as used in *fit*.

In [9]:
batch_size = 32
model.fit(x_train,y_train,batch_size = batch_size, epochs = 5)
score = model.evaluate(x_test,y_test,verbose = 0)

Epoch 1/5
281/281 [==============================] - 7s 24ms/step - loss: 1.4074 - accuracy: 0.6826
Epoch 2/5
281/281 [==============================] - 6s 20ms/step - loss: 0.7744 - accuracy: 0.8200
Epoch 3/5
281/281 [==============================] - 5s 19ms/step - loss: 0.5520 - accuracy: 0.8664
Epoch 4/5
281/281 [==============================] - 6s 20ms/step - loss: 0.4244 - accuracy: 0.8926
Epoch 5/5
281/281 [==============================] - 6s 20ms/step - loss: 0.3408 - accuracy: 0.9130


If you have done everything as specified, in particular set the random seed as we did above, your test accuracy should be around 80% 

In [10]:
score[1]

0.7951914668083191